# Fitting a Logistic Regression Model - Lab

## Introduction

In the last lesson you were given a broad overview of logistic regression. This included an introduction to two separate packages for creating logistic regression models. In this lab, you'll be investigating fitting logistic regressions with `statsmodels`. For your first foray into logistic regression, you are going to attempt to build a model that classifies whether an individual survived the [Titanic](https://www.kaggle.com/c/titanic/data) shipwreck or not (yes, it's a bit morbid).


## Objectives

In this lab you will: 

* Implement logistic regression with `statsmodels` 
* Interpret the statistical results associated with model parameters

## Import the data

Import the data stored in the file `'titanic.csv'` and print the first five rows of the DataFrame to check its contents. 

In [1]:
# Import the data
import pandas as pd

df = pd.read_csv('titanic.csv')
df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Define independent and target variables

Your target variable is in the column `'Survived'`. A `0` indicates that the passenger didn't survive the shipwreck. Print the total number of people who didn't survive the shipwreck. How many people survived?

In [2]:
# Total number of people who survived/didn't survive
y = df['Survived']
all_in = df['Survived'].value_counts()
print(f' Number of survivors: {all_in.get(1,0)}')
print(f' Number of those who never survived: {all_in.get(0,0)}')



 Number of survivors: 342
 Number of those who never survived: 549


Only consider the columns specified in `relevant_columns` when building your model. The next step is to create dummy variables from categorical variables. Remember to drop the first level for each categorical column and make sure all the values are of type `float`: 

In [3]:
# Create dummy variables
relevant_columns = ['Pclass', 'Age', 'SibSp', 'Fare', 'Sex', 'Embarked', 'Survived']
dummy_dataframe = pd.get_dummies(df[relevant_columns], drop_first=True, dtype=int)

dummy_dataframe.shape

(891, 8)

Did you notice above that the DataFrame contains missing values? To keep things simple, simply delete all rows with missing values. 

> NOTE: You can use the [`.dropna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html) method to do this. 

In [4]:
# Drop missing rows
dummy_dataframe = dummy_dataframe.dropna()
dummy_dataframe.shape

(714, 8)

Finally, assign the independent variables to `X` and the target variable to `y`: 

In [5]:
# Split the data into X and y
y = dummy_dataframe['Survived']
X = dummy_dataframe.drop(columns='Survived')

print(X)
print(y)

     Pclass   Age  SibSp     Fare  Sex_male  Embarked_Q  Embarked_S
0         3  22.0      1   7.2500         1           0           1
1         1  38.0      1  71.2833         0           0           0
2         3  26.0      0   7.9250         0           0           1
3         1  35.0      1  53.1000         0           0           1
4         3  35.0      0   8.0500         1           0           1
..      ...   ...    ...      ...       ...         ...         ...
885       3  39.0      0  29.1250         0           1           0
886       2  27.0      0  13.0000         1           0           1
887       1  19.0      0  30.0000         0           0           1
889       1  26.0      0  30.0000         1           0           0
890       3  32.0      0   7.7500         1           1           0

[714 rows x 7 columns]
0      0
1      1
2      1
3      1
4      0
      ..
885    0
886    0
887    1
889    1
890    0
Name: Survived, Length: 714, dtype: int64


## Fit the model

Now with everything in place, you can build a logistic regression model using `statsmodels` (make sure you create an intercept term as we showed in the previous lesson).  

> Warning: Did you receive an error of the form "LinAlgError: Singular matrix"? This means that `statsmodels` was unable to fit the model due to certain linear algebra computational problems. Specifically, the matrix was not invertible due to not being full rank. In other words, there was a lot of redundant, superfluous data. Try removing some features from the model and running it again.

In [6]:
# Build a logistic regression model using statsmodels
import statsmodels.api as sm
model = sm.Logit(y,X)
results = model.fit()

Optimization terminated successfully.
         Current function value: 0.507421
         Iterations 6


## Analyze results

Generate the summary table for your model. Then, comment on the p-values associated with the various features you chose.

In [7]:
# Summary table
print(results.summary())

                           Logit Regression Results                           
Dep. Variable:               Survived   No. Observations:                  714
Model:                          Logit   Df Residuals:                      707
Method:                           MLE   Df Model:                            6
Date:                Fri, 23 May 2025   Pseudo R-squ.:                  0.2487
Time:                        11:15:47   Log-Likelihood:                -362.30
converged:                       True   LL-Null:                       -482.26
Covariance Type:            nonrobust   LLR p-value:                 5.843e-49
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Pclass         0.0319      0.093      0.345      0.730      -0.150       0.213
Age            0.0019      0.006      0.330      0.742      -0.010       0.014
SibSp         -0.2699      0.116     -2.324      0.0

In [13]:
# Your comments here
'''
Sex had the most influence in terms of survival with a coefficient of -2.1321. 
Features that were statistically significant were Sex, Fare, SibSp.

Survival was more likely for:

Females — men had much lower odds of surviving.

Passengers who paid higher fares.

Those with fewer siblings/spouses aboard.

Not significant: Age, passenger class, and embarkation point didn't strongly affect survival when other factors were considered
'''



"\nSex had the most influence in terms of survival with a coefficient of -2.1321. \nFeatures that were statistically significant were Sex, Fare, SibSp.\n\nSurvival was more likely for:\n\nFemales — men had much lower odds of surviving.\n\nPassengers who paid higher fares.\n\nThose with fewer siblings/spouses aboard.\n\nNot significant: Age, passenger class, and embarkation point didn't strongly affect survival when other factors were considered\n"

## Level up (Optional)

Create a new model, this time only using those features you determined were influential based on your analysis of the results above. How does this model perform?

In [11]:
# Your code here
X_2 = dummy_dataframe[['Sex_male', 'Fare', 'SibSp']]
model_2 = sm.Logit(y,X_2)
results_2 = model_2.fit()
print(results_2.summary())

Optimization terminated successfully.
         Current function value: 0.511829
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:               Survived   No. Observations:                  714
Model:                          Logit   Df Residuals:                      711
Method:                           MLE   Df Model:                            2
Date:                Fri, 23 May 2025   Pseudo R-squ.:                  0.2422
Time:                        11:16:45   Log-Likelihood:                -365.45
converged:                       True   LL-Null:                       -482.26
Covariance Type:            nonrobust   LLR p-value:                 1.858e-51
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Sex_male      -1.8566      0.144    -12.931      0.000      -2.138      -1.575
Fare           0.0201      0.

In [12]:
# Your comments here
''' 
- The model is statistically significant and fits better than a null model.

- Sex and Fare are strong predictors of survival.

- Being male and having more siblings/spouses lowers survival odds.

- Paying a higher fare improves survival chances — perhaps reflecting class or cabin location.
'''

' \n- The model is statistically significant and fits better than a null model.\n\n- Sex and Fare are strong predictors of survival.\n\n- Being male and having more siblings/spouses lowers survival odds.\n\n- Paying a higher fare improves survival chances — perhaps reflecting class or cabin location.\n'

## Summary 

Well done! In this lab, you practiced using `statsmodels` to build a logistic regression model. You then interpreted the results, building upon your previous stats knowledge, similar to linear regression. Continue on to take a look at building logistic regression models in Scikit-learn!